In [ ]:
REFERENCE_LOC = '/scripts/data/polygons/digi.shp'
fieldname = 'TYPE'
RASTERSTACK = '/data/Hdd1/SUMATRA.vrt'
OUTPUTFOLDER = '/data/generated'

mode = 'ne'
scl = 400/2

In [ ]:
%%bash
earthengine authenticate --quiet

In [ ]:
%%bash
earthengine authenticate --authorization-code=

In [ ]:
import ee

if mode == 'ee':    
    try:
      ee.Initialize()
      print('The Earth Engine package initialized successfully!')
    except ee.EEException as e:
      print('The Earth Engine package failed to initialize!')
    except:
        print("Unexpected error:", sys.exc_info()[0])
        raise

    ee.Initialize()

In [ ]:
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')
vh = sentinel1
  // Filter to get images with VV and VH dual polarization.
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
  // Filter to get images collected in interferometric wide swath mode.
  .filter(ee.Filter.eq('instrumentMode', 'IW'));
vh = sentinel1.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW'))
vhDescending = vh.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'));

composite = ee.Image.cat([
  vhDescending.select('VH').mean(),
  ee.ImageCollection(vhDescending.select('VV').merge(vhDescending.select('VV'))).mean(),
  vhDescending.select('VH').mean()
]).focal_median()


In [ ]:
import numpy as np
from shapely import geometry
from fiona.crs import from_epsg
from fiona import collection
import fiona
import utm
import subprocess
from osgeo import gdal

In [ ]:
#src = fiona.open('/run/media/ron/Hdd1/reference_data/digi.shp')
src = fiona.open(REFERENCE_LOC)
schema = { 'geometry': 'Polygon', 'properties': { 'name': 'str' } }

In [ ]:
with collection(OUTPUTFOLDER + "/some35.geojson", "w", "GeoJSON", schema=schema) as output:
    for f in src:
        polygon = f['geometry']['coordinates'][0]#f[0]['geometry']['coordinates']
        P = geometry.Polygon([p for p in polygon])
        C = P.centroid
        U = utm.from_latlon(C.y, C.x)
        P = geometry.point.Point(U) 
        B = P.centroid.buffer(scl, cap_style=3)        
        eh = geometry.mapping(B)
        wgsbuffs = []
        for i, hm in enumerate(eh['coordinates'][0]):
            try:
                coords = utm.to_latlon(hm[0], hm[1], U[2], U[3])
                wgsbuffs.append(coords)
            except:
                print('ha')               

        if len(wgsbuffs) == 5:
            P2 = geometry.Polygon([(p[1],p[0]) for p in wgsbuffs])    
            output.write({
                'properties': {
                    'name': 'bla'
                },
                'geometry': geometry.mapping(P2)
                })
            
            newpoly = [[w[1],w[0]] for w in wgsbuffs]
            
            outraster = 'Y' + str(int(C.y)) + 'X' + str(int(C.x))
            if mode == 'ee':
                task = ee.batch.Export.image.toDrive(composite, 'hm/' + outraster, scale=10, region=newpoly)
                task.start()
                
            elif mode == 'ne':                
                ulx = np.min(np.array(newpoly),0)[0]
                uly = np.max(np.array(newpoly),0)[1]
                lrx = np.max(np.array(newpoly),0)[0]
                lry = np.min(np.array(newpoly),0)[1]
                print(ulx)
                print(uly)
                print(lrx)
                print(lry)
                # use coordinates and gdal_translate to reduce raster stack
                subprocess.call(['gdal_translate', '-projwin', str(ulx), str(uly), str(lrx), str(lry), RASTERSTACK, OUTPUTFOLDER + '/' + outraster + '.tif'])

In [ ]:
import glob
rasterfilenames = glob.glob(OUTPUTFOLDER + '/*.tif')

In [ ]:
for r in rasterfilenames:
    referencerasterloc = r
    outputname = r[:-4] + '_y.tif'    

    Shp_src = ogr.Open(REFERENCE_LOC)
    Ras_src = gdal.Open(referencerasterloc)
    rasterdriver = gdal.GetDriverByName('GTiff')

    new_raster = rasterdriver.Create(outputname, Ras_src.GetRasterBand(1).XSize, Ras_src.GetRasterBand(1).YSize, 1, gdal.GDT_Byte)
    new_raster.SetProjection(Ras_src.GetProjection())
    new_raster.SetGeoTransform(Ras_src.GetGeoTransform())

    Shp_lyr = Shp_src.GetLayer()
    gdal.RasterizeLayer(new_raster, [1], Shp_lyr, None, None, [1], ['ATTRIBUTE='+fieldname])